# Goal of the project

The goal of this project is to control a 2D quadrotor to perform acrobatic moves. There are 4 parts of the project, where you will build controllers of increasing complexity. The last part will lead to the implementation of the iterative LQR (iLQR) algorithm.

## Instructions
Answer all the questions in the 4 parts below. You will need to submit:
1. A report (pdf format only - every other format will be rejected) answering all the questions that do not request code. DO NOT include code in the report.
2. One (or several) Jupyter notebook(s) containing all the code used to answer the questions. The notebook(s) should be runnable as is.

## 2D quadrotor

The quadrotor is depicted in the following figure <img src='quadrotor.png' width="300">
The quadrotor model is written as
$$\begin{align} 
\dot{x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $x$ is the horizontal and $y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $z = [x, v_x, y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines useful constants (mass, length, gravity, etc) and functions to simulate and animate the quadrotor as shown below.

In [1]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

import quadrotor

from quadrotor import MASS, INERTIA, LENGTH, GRAVITY, DELTA_T, NUMBER_STATES as N_STATES, NUMBER_CONTROLS as N_CONTROLS

G = MASS * GRAVITY

## Part 1 - Setting up
1. Discretize the system dynamics using the method seen in class - write the time discretization step as $\Delta t$ (use symbols not numbers for the mass, etc)
2. Assume that the robot starts at an arbitrary position $x(0) = x_0$, $y(0) = y_0$ and $\theta(0) = 0$ with 0 velocities. Compute $u_1^*$ and $u_2^*$ such that the robot stays at this position forever after (you may test your answer using the simulation below).
3. Analyzing the system dynamics, is it possible to move in the x direction while keeping $\theta = 0$? Explain why.
4. Analyzing the system dynamics, is it possible to have the system at rest with $\theta = \frac{\pi}{2}$ (i.e. have the quadrotor in a vertical position)? Explain why.

1. descrtize the system:
- $x_{n+1}=x_n+\dot{x}_n\Delta t$
- $\dot{x}_{n+1}=\dot{x}_n-[(u_1+u_2)\sin\theta/m]\Delta t$
- $y_{n+1}=y_n+\dot{y}_n\Delta t$
- $\dot{y}_{n+1}=\dot{y}_n+[(u_1+u_2)\cos\theta/m-g]\Delta t$
- $\theta_{n+1}=\theta_n+\omega_n\Delta t$
- $\omega_{n+1}=\omega_n+[r(u_1-u_2)/I]\Delta t$
   
2. keep $v_x=v_y=0\Rightarrow u_1=u_2=mg/2$

3. possible, with an initial horizontal velocity 

4. impossilbe, $y$ and $v_y$ are out of control



In [2]:
# stay at the original position

def controller(state, i):
    return np.array([G / 2, G / 2])


horizon_length = 1000
z0 = np.zeros([N_STATES,])

t, state, u = quadrotor.simulate(z0, controller, horizon_length, disturbance = False)
quadrotor.animate_robot(state, u)

<source type="video/mp4" src="data:video/mp4;base64,AAAAIGZ0eXBNNFYgAAACAE00ViBpc29taXNvMmF2YzEAAAAIZnJlZQAAI7RtZGF0AAACoQYF//+d
3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MSAtIEguMjY0L01QRUctNCBBVkMgY29kZWMg
LSBDb3B5bGVmdCAyMDAzLTIwMjAgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwg
LSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMg
bWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5n
ZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEg
ZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0yNCBsb29rYWhlYWRfdGhy
ZWFkcz00IHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVy
YXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9h
ZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBr
ZXlpbnQ9MjUwIGtleWludF9taW49MTAgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xv
b2thaGVhZD00MCByYz1jcmYgbWJ0cmVlPTEgY3JmPTIzLjAgcWNvbXA9MC42MCBxcG1pbj0wIHFw
bWF4PTY5IHFwc3RlcD00IGlwX3JhdGlvPTEuNDAgYXE9MToxLjAwAIAAABMRZYiEABH//veIHzLL
afk613IR560urR9Q7kZxXqS9/iAAAAMAAAMAAAMACpJYniKH8XjN/AAAAwAAt4AXsAwgCqgMKj6I
dndnTzAIiR6+OnDbVkTjpPexGHLjoXY30hkXEhSE9kEKIJNbrApCI4G73ihHZi4T1K1iog5Wc0p6
pLHThCwCmnYhSqYpYbTo2n5tZjl9Sersc3rmxTCU1aG5ejWlf2ZiBPtJ6vuZEf6FCWPekrjf3iOx
g9nYeAC3//ekaJ6Uup8pEnX74AtF5miPCXmGYp1VgIlMhblKE1f5XLlxb1HUXmuG+fEgDCAJNggr
SWMoFVS8OFpaDDMvSO64pmzh6vHDoKtfmcA4i12uUsh7rwNZe/BI7LdbaaCmFlaKG6p3WlZZBOgL
c6Xh6KGqBk+DXTIvzmnGdaE/vYVZrpQqE5tWlH1wLzpsCVXhaMhwyKlIHYXCzI57hN1GXk5AJs78
kL3L0uFeE8COidDOFd4yzkoe/7l2h6wEKOz7uFgABdAAABgT/XL0eR8k6J3lRIGcTlhh/LgQemvB
9LeOOPbS/io9bEKpktMzUsr2rH7n7gn+AW5ayX7+OPXTsJWi5Af6cl+Qnz0uW+DYBLS5p2thXpov
CDk56tcwW0pgAmn5ff2hXXmI9LQE3pWGz9avZjnRjG4VNxvRD6MJBuoGFZz0TZh4HejXX+Xr6AY1
NtLhG5Dt0FFT7heDy2T/J56stCiLolYaEC4Lsf3Ba/m1qzWfMEIyANp7aDPKoMHvT9ZjAKv7FBZX
Xxf0d3g8dTn4pTf1GQPGRmfytUQyQGmaJAtChL15m3Bfb2TtW1+3gzSZ1S1Rg4BH5ZPE24z39rF9
EoPALouyjq/tksmA0xnvlOq2NQQ7nQGy++HnWaMv68jeZLfIspnHkGYLlaiHH8+7/gxXkppoqbbV
zGkbOLAlUxuB3Hkcp5ozwRFJEkffYokAAAMAAAMAAAMAAAU6K2WUArTaxeJ+FAI6eDOlC41QrK3f
HaPOA6Tkh7/DwxsYaebi6Zz/tc3X0+AhNf+/w/jFcrRNyaPOOEcBzxEuEPILst0ElfjGP5v3AH5X
yXBfc7WidnYCZrsawbrLqWTm4QgoAdRgLV0U555i/86kNOiHr46Nw6torKk1r7mNtZ/T/Xe6+E7+
EjZHG90CyaEPaGS3j/5mYHg3MA5rbEmvPGuisLgNZSWC1YVoTVeDeOdKGR4E2btWR6qv3qENxUyo
8iZFN4bCV4sg7yVTfCv+Q1WwrD7TEkA1RBJAbcuHVY/8Cr9g1yT55JNil2P/LNaXYUIQw3dbuA0A
Pk8XHLIUBKF97ZANsF7aGYwAx5/lXZ4VehoU4xcE8RhlLFM4cpp+d+Knot6APk5kCQAAbt3cswf9
8VQgESJkJJAA+SWSIQAb8pg7QEFCMiI9fXlA5RchbcN6cnpCg4tPCQQUQzEYuMw5TSwFBoseTShV
DvUG8HqMeInlNnSc35dg6qXdjP7Eua7hazlhmTTc0VS9ZoYRvwPFL4tfuY06N4iDUZjIQbg7U70s
URUHGExDjY1GOP+WaHftiuowiY2wZc2zZBbJN6IPGA+CheMfDJvd1NLOEP0ozXYw03vnZs/W9gCx
b1e23z/vXgKv7enJbogTUHDv4z1gABhP1/SXYHRQsyIRRJnZQHPe5lQLlhk7ZJsEYIzSVMlJeek+
SyiSt0lurw+wWSMU6ek23ZY+4t+OO+LOBc440+D65iGCaY12Mo62QSRuIyOPrpSHL1Ms/r+T68E5
hrb4xy99bkV2XzIsD5hKbX839f5IRid5vjrFQYrfEsU5YFsMypirxPu/i9/x3QEKO7zbBAbohpDn
Rqx2iDHSG7RQWmLnXQao9tuKoX9+DcfdHCcbLASU5acP+JRtitWU4P3fBwAAHWOaiO1fjyAAAkrf
Z1QJmw0ALFXyG1Un0dEgLINfV32Yq7TaBRXTQQKBEz55TDdwIF3UvohMgPBuUJmkDcObvMVcddXx
eifEVsDsU9Nn/MUD9R1OE/bOkNGJBj4f9nWmbCCypjajPzWo8qeG+rH0qlbyGq77+9ZAcoGBepA0
as8eiWzyLk83+wVHDlJe/iAj/BwBRSkUU7MTfQsKsRXxKXSg9iE+WSTCcyM/YbnHvDvg/sBcQ/VM
UzPrUi9i5XyjmVVGPFCLtFCDuuQjchgDAOzr/sXcQD6SDLoYPUy+5HFTZ8mwEVFX8ZZu0TxmAX8e
dbnshBO50FdZoNFJw+wGeWDj50zeAKYfLwmYhP8ovSHn9H6B6ynNzhbb0QsftZZCqZ62tbdul2K2
mw/Y2azeFc7ocd8AqksWZg2OUD9+f5rbziGt7dmOY3p8RutUNBRtzKT4o5/oawBOZOBT40dNeEq9
0j+Iu7RImYsLt32avaSYyDqNuTRKQ0Sae25sOUbSyrv2Hn6lg+/WQiZydyxlPZPHyhsHoXUmAWan
I9YwVvVvPsCD6+eB5JENXrrblfpAc165f3UZDJ9ukkUP7L6WENxsjv7PNcQm6ak6z2a1mfPfMm/P
O0BFqEV6HdpjYnQuOODq3S4+ak+Q23mgbMF6fI3tVZPcoRjkgk/IjQ+47ABABBmoxNvEQ6NZQ5a5
EjpxWardf86D0AnlFTaNtJz5p5G75y+t8+p8xl2q5ULqKw6YWs2ZDc3FoCajKd0tbq4MS/a2APtN
XwAYSevBNuELKVoIm1PtV8RsMSqICPDbpglLZ/NoM1YABPBFKzPx3GF0TPfS2S99YNULHYlfi1mk
Z73vavMpQ5ACq0hf3RLLgNg0VBsccOEyvQT4n0YaKx7MDA7AlG/VRZOvyIneaJ5Tsa3jMkGJ59j9
eaw4Qq85OxV7XX/u4rWp3bLO+Sxp/IYKKZ2lUaUYURbZLwd6UNODadPlzCloXqF6LTaNMvwgyMsY
igYdC6tHWIQQYoEFkxRze

## Part 2 - LQR to stay in place
Now that we have $u^*$ capable of keeping the robot at rest, we can design a simple controller that ensures that the robot stays in place even when pushed around by random disturbances (e.g. due to the wind). Our task here will be to design a LQR controller that keeps the robot at a predefined position. Since the dynamics is not linear, we need to compute a linear approximation of it.
1. Linearize the dynamics at an arbitrary operating point $z^*$, $u^*$ and write the linearized system dynamics using the variables $\bar{z}_n = z_n - z^*$ and $\bar{u}_n = u_n - u^*$.
2. Write a function ```get_linearization(z, u)``` that returns the matrices A and B given a state $z$ and a control $u$ (use the constants defined in the ``quadrotor.py`` module).
3. Using the linearized dynamics, we can design an infinite horizon LQR controller of the form, $\hat{u} = K \bar{z}$ to stabilize the resting point. Write the equations of the controller in the original coordinates $u$ as a function of $z$.
4. Design an infinite-horizon LQR controller that stabilizes the origin $z=0$ and test it using the simulator below.
5. Explain your intended design in the report, including the cost function and found control law. In particular, verify that it can handle perturbations by calling the ```simulate``` function with ```disturbance = True``` (when setting disturbance to ``True``, the simulator will generate a random perturbation every 1 second). Simulate your controller for 10 seconds, plot the state evolution and show the animation (include the plots in your report).

1. linearize the system:

- $x=x^*+1\cdot\bar{x}+\Delta t\cdot\bar{v_x}$
- $v_x=v_x^*+1*\bar{v_x}-\frac{\cos\theta^*(u_1^*+u_2^*)\Delta t}{m}\cdot\bar{\theta}-\frac{\sin\theta^*\Delta t}{m}\cdot(\bar{u_1}+\bar{u_2})$
- $y=y^*+1\cdot\bar{y}+\Delta t\cdot\bar{v_y}$
- $v_y=v_y^*+1\cdot\bar{v_y}-\frac{\sin\theta^*(u_1^*+u_2^*)\Delta t}{m}\cdot\bar\theta+\frac{\cos\theta^*\Delta t}{m}\cdot(\bar{u_1}+\bar{u_2})$
- $\theta=\theta^*+1\cdot\bar{\theta}+\Delta t\cdot\bar{\omega}$
- $\omega=\omega^*+1\cdot\bar{\omega}+\Delta t\cdot\frac{r}{I}\cdot(\bar{u_1}-\bar{u_2})$

2. in matrix form:

$z=\left[\begin{matrix}x^* \\ v_x^* \\ y^* \\ v_y^* \\ \theta^* \\ \omega^*\end{matrix}\right]+\left[\begin{matrix}1 & \Delta t & 0 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 & -\frac{\cos\theta^*(u_1^*+u_2^*)\Delta t}{m} & 0 \\ 0 & 0 & 1 & \Delta t & 0 & 0 \\ 0 & 0 & 0 & 1 & -\frac{\sin\theta^*(u_1^*+u_2^*)\Delta t }{m} & 0 \\ 0 & 0 & 0 & 0 & 1 & \Delta t \\ 0 & 0 & 0 & 0 & 0 & 1\end{matrix}\right]\left[\begin{matrix}\bar x \\ \bar{v_x} \\ \bar y \\ \bar{v_y} \\ \bar\theta \\ \bar\omega\end{matrix}\right]+\left[\begin{matrix}0 & 0 \\ -\frac{\sin\theta^*\Delta t}{m} & -\frac{\sin\theta^*\Delta t}{m}\\ 0 & 0 \\ \frac{\cos\theta^*\Delta t}{m} & \frac{\cos\theta^*\Delta t}{m} \\ 0 & 0 \\ \frac{r}{I}\Delta t & -\frac{r}{I}\Delta t\end{matrix}\right]\left[\begin{matrix}\bar{u_1} \\ \bar{u_2}\end{matrix}\right]$

3. $u_n=K_n(z_n-z^*)+u^*$


4. assuming that control is cheap:

$Q=diag(100,1,100,1,100,1)$

$R=diag(1,1)$

In [3]:
def get_linearization(z, u):
    A = np.mat([
        [1,DELTA_T,0,0,0,0],
        [0,1,0,0,-np.cos(z[4])*(u[0]+u[1])*DELTA_T/MASS,0],
        [0,0,1,DELTA_T,0,0],
        [0,0,0,1,-np.sin(z[4])*(u[0]+u[1])*DELTA_T/MASS,0],
        [0,0,0,0,1,DELTA_T],
        [0,0,0,0,0,1]
    ])
    B = np.mat([
        [0,0],
        [-np.sin(z[4])*DELTA_T/MASS,-np.sin(z[4])*DELTA_T/MASS],
        [0,0],
        [np.cos(z[4])*DELTA_T/MASS,np.cos(z[4])*DELTA_T/MASS],
        [0,0],
        [LENGTH/INERTIA*DELTA_T,-LENGTH/INERTIA*DELTA_T]
    ])
    return A, B


Q = np.mat(np.diag([100,1,100,1,100,1]))
R = np.mat(np.diag([1,1]))

def solve_LQR(z, u, n):
    A, B = get_linearization(z, u)
    p = np.zeros((n + 1, N_STATES, N_STATES))
    p[-1] = Q
    k = np.zeros((n, N_CONTROLS, N_STATES))

    for i in range(n - 1, -1, -1):
        k[i] = -1 * (B.T * p[i + 1] * B + R).I * B.T * p[i + 1] * A
        p[i] = Q + A.T * p[i + 1] * A + A.T * p[i + 1] * B * k[i]
    
    return k

horizon_length = 1000
z0 = np.zeros([N_STATES,])

z_target = z0
u_target = np.array([G / 2, G / 2])
k = solve_LQR(z_target, u_target, 100)

def controller(state, i):
    return (k[i % 100] * np.mat(state - z_target).T).A1 + u_target


t, state, u = quadrotor.simulate(z0, controller, horizon_length, disturbance = True)
quadrotor.animate_robot(state, u)

Your browser does not support the video tag.

## Part 3 - following a trajectory using linearized dynamics
Now we want to follow a given trajectory leveraging a linearized version of the dynamics to design LQ controllers.
1. Assume that we want to follow a circle of radius 1 centered at (0,0) while keeping an orientation $\theta=0$, how does the linearization of the dynamics change along the desired trajectory? Why?
2. Design a tracking controller (using an LQ design with linear approximations) to follow this desired trajectory. Explain your design in the report. 
3. Test the tracking controller with the simulation (with and without the perturbations) and verify that you can indeed track the (x,y) trajectory very well. Are you able to also track $\theta$? (Explain) 
4. Analyze your results (including plots of the states, controls, etc). What benefits and issues do you see with this approach?
5. Is it possible to do the same thing while keeping a desired orientation of $\theta = \frac{\pi}{4}$? What might influence the results in this case?

1. assuming that the period is $T=n\Delta t$ ，and at the timestep $i$ we have
   - $x^*=\cos(\frac{2\pi}{T}\cdot i\Delta t)$
   - $y^*=\sin(\frac{2\pi}{T}\cdot i\Delta t)$
   - $\theta^*=0$


In [4]:
def get_target_trajectory(horizon_length, T):
    z = np.empty([N_STATES, horizon_length])
    for i in range(horizon_length):
        phase = 2*np.pi/T*i*DELTA_T
        state = np.zeros([N_STATES, ])
        state[0] = np.cos(phase)
        state[1] = -2*np.pi/T*np.sin(phase)
        state[2] = np.sin(phase)
        state[3] = 2*np.pi/T*np.cos(phase)
        z[:, i] = state
    return z

Q = np.mat(np.diag([100,1,100,1,100,1]))
R = np.mat(np.diag([0.01, 0.01]))


def get_linearization(z, u):
    A = np.mat([
        [1,DELTA_T,0,0,0,0],
        [0,1,0,0,-np.cos(z[4])*(u[0]+u[1])*DELTA_T/MASS,0],
        [0,0,1,DELTA_T,0,0],
        [0,0,0,1,-np.sin(z[4])*(u[0]+u[1])*DELTA_T/MASS,0],
        [0,0,0,0,1,DELTA_T],
        [0,0,0,0,0,1]
    ])
    B = np.mat([
        [0,0],
        [-np.sin(z[4])*DELTA_T/MASS,-np.sin(z[4])*DELTA_T/MASS],
        [0,0],
        [np.cos(z[4])*DELTA_T/MASS,np.cos(z[4])*DELTA_T/MASS],
        [0,0],
        [LENGTH/INERTIA*DELTA_T,-LENGTH/INERTIA*DELTA_T]
    ])
    return A, B
def solve_LQR_with_trajectory(z, u, n):
    p = np.zeros((n + 1, N_STATES, N_STATES))
    p[-1] = Q
    k = np.zeros((n, N_CONTROLS, N_STATES))
    qt = -1 * Q * z
    pt = np.zeros((n + 1, N_STATES, 1))
    pt[-1] = qt[:, -1]
    kt = np.zeros((n, N_CONTROLS, 1))
    for i in range(n - 1, -1, -1):
        A, B = get_linearization(z[:, i], u)
        k[i] = -1 * (B.T * p[i + 1] * B + R).I * B.T * p[i + 1] * A
        p[i] = Q + A.T * p[i + 1] * A + A.T * p[i + 1] * B * k[i]
        kt[i] = -1 * (B.T * p[i + 1] * B + R).I * B.T * pt[i + 1]
        pt[i] = qt[:, i] + A.T * pt[i + 1] + A.T * p[i + 1] * B * kt[i]
    return k, kt

horizon_length = 1000
z0 = np.array([1, 0, 0, 0, 0, 0])

z_target = get_target_trajectory(horizon_length + 1, horizon_length * DELTA_T)
u_target = np.array([G / 2, G / 2])
k, kt = solve_LQR_with_trajectory(z_target, u_target, horizon_length)


def controller(state, i):
    return (k[i] * np.mat(state).T).A1 + kt[i].squeeze() + u_target


t, state, u = quadrotor.simulate(z0, controller, horizon_length, disturbance = True)
quadrotor.animate_robot(state, u)

Your browser does not support the video tag.

## Part 4 - iterative LQR
Now we would like to do more complicated motions with the robot, like a flip. In this case, we do not have a prescribed trajectory but we would like to compute a locally optimal trajectory while we optimize the controller. We will use the *iterative LQR* algorithm to solve this problem.
### Task 1 - reaching a vertical orientation
In the first task, we want the robot to reach a vertical orientation $\theta = \frac{\pi}{2}$ at the location $x=3$ and $y=3$ at time $t=5$ starting from $z_0=0$. During the rest of the motion, the robot should try and stay close to the origin. It should also try to keep its control $u$ close to the control needed to keep the robot at rest. We want to make sure the robot reaches the origin $z=0$ at the end of the movement. 
1. Find a time-varying cost function that promotes such a behavior (use only quadratic/linear terms for simplicity). Assume $T=10$ seconds.
2. Write a function ```compute_cost(z,u, horizon_length)``` that returns the cost of a trajectory z with control trajectory u (using the cost function you wrote in question 1).
3. Compute the quadratic approximation of your cost function along an arbitrary trajectory of states $z_n$ with control trajectory $u_n$ (this is not just your cost function!)
4. Write a function ```get_quadratic_approximation_cost(z, u, horizon_length)``` that returns the quadratic approximation (Hessian matrices and Jacobians) of the cost function when approximated along the trajectory z with control trajectory u.
5. Write the iLQR algorithm that solves the problem using the functions written above. DO NOT FORGET the line search step at each iteration. For the line search, start with $\alpha = 1.$ and decrease it by half when the cost does not improve (you can stop when $\alpha < 0.01$).
6. Test the algorithm using as initial guess $u$ such that the robot is at rest (using the results of Part 1.2). Analyze your results (probably you will need to "tune" your cost function), plot the initial and final state and control trajectories, show the animation. Use the simulation without perturbations for simplicity. 
7. What benefits and issues do you see with this approach?
### Task 2 - doing a full flip
In the second task, we want the robot to do a full flip, trying to reach the upside-down state $x=1.5$, $y=3$ and $\theta = \pi$ at $t=5$ and upright state $x=3$, $y=0$ and $\theta = 2\pi$ at $T=10$.
8. Use iLQR (and a new cost function) to get the quadrotor to perform the task. Analyze your results. 
9. What benefits and issues do you see with this approach? Could you run the resulting controller on a real robot?

1. Quadratic Cost:

$J=\sum_{i=1}^N(\bar z_i^TQ_i\bar z_i+\bar u_i^TR_i\bar u_i)$ ,with $\bar z_i=z_i-z^*_i,\bar u_i=u_i-u^*_i$

$N=1000$

$z_{500}^*=[3,0,3,0,\frac{\pi}{2},0], z_{1000}^*=[0,0,0,0,0,0]$

2. Flip

$Q_i=O,Q_{500}=Q_{1000}=100*(diag(10,1,10,1,1000,1))$

3. Vertical

$Q_i=O,Q_{500}=Q_{1000}=100*(diag(1000,1,1000,1,100,1))$

$R_i=diag(0.1, 0.1)$

In [10]:
MAX_ITERATION = 50
TOLERENCE = 0.001
THRESHOLD = 0.01

Q = np.mat(np.diag([1,1,1,1,1,1]))
# Flip
Q_KEY = np.mat(np.diag([10,1,10,1,1000,1])*100)
#Vertical
# Q_KEY = np.mat(np.diag([1000,1,1000,1,1000,1])*100)
R = np.mat(np.diag([0.1, 0.1]))

#Vertical
# waypoints = {
#     499: np.array([3,0,3,0,np.pi/2,0])
# }

# baseline = {
#     0: np.array([0, 0, 0, 0, 0, 0]),
# }

# Flip
waypoints = {
    499: np.array([1.5,0,3.5,0,2*np.pi,0]),
#     499: np.array([1.5,0,3,0,np.pi,0]),
    
    #999: np.array([3, 0, 0, 0, 2*np.pi, 0])
}

baseline = {
    0: np.array([0, 0, 0, 0, 0, 0]),
    600: np.array([3, 0, 0, 0, 2*np.pi, 0])
}

u_target = np.array([G / 2, G / 2])

def compute_cost(z, u, horizon_length):
    j = 0
    base = baseline[0]
    for i in range(horizon_length):
        if i in baseline:
            base = baseline[i]
        u_bar = np.mat(u[:, i] - u_target).T
        z_bar = np.mat(z[:, i] - waypoints[i]).T if i in waypoints else np.mat(z[:, i] - base).T
        q = Q_KEY if i in waypoints else Q
        j += (z_bar.T*q*z_bar + u_bar.T*R*u_bar)[0,0]
    return j

def get_quadratic_approximation_cost(z, u, horizon_length):
    return a, b, q, r
    

def solve_ilqr(z0, u, n):

    def backward(z, u):
        p = np.zeros((n + 1, N_STATES, N_STATES))
        p[-1] = Q
        k = np.zeros((n, N_CONTROLS, N_STATES))
        qt = np.zeros((n, N_STATES, 1))
        base = baseline[0]
        for i in range(n):
            if i in baseline:
                base = baseline[i]
            qt[i] = (-1 * Q * np.mat(base).T)
        for key, z_ in waypoints.items():
            qt[key] = (-1 * Q_KEY * np.mat(z_).T)
        pt = np.zeros((n + 1, N_STATES, 1))
        pt[-1] = qt[-1]
        kt = np.zeros((n, N_CONTROLS, 1))
        rt = - R * np.mat(u_target).T
        for i in range(n - 1, -1, -1):
            q = Q_KEY if i in waypoints else Q
            A, B = get_linearization(z[:, i], u[:, i])
            k[i] = -1 * (B.T * p[i + 1] * B + R).I * B.T * p[i + 1] * A
            p[i] = q + A.T * p[i + 1] * A + A.T * p[i + 1] * B * k[i]
            kt[i] = -1 * (B.T * p[i + 1] * B + R).I * (B.T * pt[i + 1] + rt)
            pt[i] = qt[i] + A.T * pt[i + 1] + A.T * p[i + 1] * B * kt[i]
        return k, kt
    
    def forward(z, u, k, kt, alpha):
        z_new = np.zeros_like(z)
        z_new[:, 0] = z[:, 0]
        u_new = np.zeros_like(u)
        
        for i in range(n):
            u_new[:, i] = u[:, i] + (k[i] * np.mat(z_new[:, i] - z[:, i]).T).A1 + alpha * kt[i].squeeze()
            z_new[:, i+1] = quadrotor.get_next_state(z_new[:, i], u_new[:, i])
            
        return z_new, u_new

    z = np.empty([N_STATES, n + 1])
    z[:, 0] = z0
    for i in range(n):
        z[:, i + 1] = quadrotor.get_next_state(z[:, i], u[:, i])
    
    alpha = 1
    it = 0
    z_opt = z
    u_opt = u

    j_pre = compute_cost(z, u, n)
    print("j_init: ", j_pre)
    while it < MAX_ITERATION:
   
        k, kt = backward(z_opt, u_opt)

        while alpha > 0.01:
            
            z, u = forward(z_opt, u_opt, k, kt, alpha)
            j = compute_cost(z, u, n)
            if j < j_pre:
                z_opt, u_opt = z, u
                break
            else:
                alpha *= 0.5
            

        print(j)

        if abs(j - j_pre) < TOLERENCE:
            break
        
        j_pre = j
        it += 1
        
    
    return z_opt, u_opt

horizon_length = 1000
z0 = np.zeros([N_STATES,])
u_init = np.zeros((N_CONTROLS, horizon_length))
for i in range(horizon_length):
    u_init[:, i] = u_target
state, u = solve_ilqr(z0, u_init, horizon_length)

quadrotor.animate_robot(state, u)

j_init:  3981733.127477435
2283123.541983926
1607901.1144696195
1051679.917658918
617000.2142748653
304716.07644891
115122.66296285886
48336.2776953475
48621.389770232236
48621.389770232236


Your browser does not support the video tag.

In [ ]:
# we can get its mass, half length (r), gravity constant
print(f'm is {quadrotor.MASS}')
print(f'r is {quadrotor.LENGTH}')
print(f'I is {quadrotor.INERTIA}')
print(f'g is {quadrotor.GRAVITY}')

# we can also get the integration step used in the simulation
print(f'dt is {quadrotor.DELTA_T}')

# we can get the size of its state and control vector
print(f'number of states {quadrotor.NUMBER_STATES} and number of controls {quadrotor.NUMBER_CONTROLS}')
print('the states are indexed as follows: x, vx, y, vy, theta, omega')

In [ ]:
# we can simulate the robot but we need to provide a controller of the following form
def dummy_controller(state, i):
    """
        the prototype of a controller is as follows
        state is a column vector containing the state of the robot
        i is the index corresponding to the time step in the horizon (useful to index gains K for e.g.)
        
        this controller needs to return an array of size (2,)
    """
    # here we do nothing and just return some non-zero control
    return 1. * np.ones([2,])




# we can now simulate for a given number of time steps - here we do 10 seconds
horizon_length = 1000
z0 = np.zeros([quadrotor.NUMBER_STATES,])
t, state, u = quadrotor.simulate(z0, dummy_controller, horizon_length, disturbance = False)

In [ ]:
# we can plot the results
plt.figure(figsize=[9,6])

plt.subplot(2,3,1)
plt.plot(t, state[0,:])
plt.legend(['X'])

plt.subplot(2,3,2)
plt.plot(t, state[2,:])
plt.legend(['Y'])

plt.subplot(2,3,3)
plt.plot(t, state[4,:])
plt.legend(["theta"])

plt.subplot(2,3,4)
plt.plot(t, state[1,:])
plt.legend(['Vx'])
plt.xlabel('Time [s]')

plt.subplot(2,3,5)
plt.plot(t, state[3,:])
plt.legend(['Vy'])
plt.xlabel('Time [s]')

plt.subplot(2,3,6)
plt.plot(t, state[5,:])
plt.legend(['omega'])
plt.xlabel('Time [s]')

# we can also plot the control
plt.figure()
plt.plot(t[:-1], u.T)
plt.legend(['u1', 'u2'])
plt.xlabel('Time [s]')

In [ ]:
# now we can also create an animation
quadrotor.animate_robot(state, u)

In [ ]:
# we can also simulate with perturbations
t, state, u = quadrotor.simulate(z0, dummy_controller, horizon_length, disturbance = True)

# we can plot the results
plt.figure(figsize=[9,6])

plt.subplot(2,3,1)
plt.plot(t, state[0,:])
plt.legend(['X'])

plt.subplot(2,3,2)
plt.plot(t, state[2,:])
plt.legend(['Y'])

plt.subplot(2,3,3)
plt.plot(t, state[4,:])
plt.legend(["theta"])

plt.subplot(2,3,4)
plt.plot(t, state[1,:])
plt.legend(['Vx'])
plt.xlabel('Time [s]')

plt.subplot(2,3,5)
plt.plot(t, state[3,:])
plt.legend(['Vy'])
plt.xlabel('Time [s]')

plt.subplot(2,3,6)
plt.plot(t, state[5,:])
plt.legend(['omega'])
plt.xlabel('Time [s]')

# we can also plot the control
plt.figure()
plt.plot(t[:-1], u.T)
plt.legend(['u1', 'u2'])
plt.xlabel('Time [s]')

quadrotor.animate_robot(state,u)